# Warsztat 3: Widoki i Podstawowe Jobsy

**Czas trwania**: 90 minut  
**Poziom**: Podstawowy/Średniozaawansowany  
**Ostatnia aktualizacja**: 2025-01-31

---

## 1. Kontekst i Cele Warsztatu

### Czego się nauczysz?
- Tworzyć widoki tymczasowe i globalne
- Tworzyć widoki materialized w Delta Lake
- Konfigurować podstawowe joby w Databricks
- Monitorować wykonanie jobów

### Wymagania wstępne
- Ukończenie Warsztatu 1 i 2
- Podstawowa znajomość SQL i PySpark
- Wyczyszczone dane z poprzedniego warsztatu

### Co będziesz implementować?
Stworzysz zestaw widoków biznesowych i skonfigurujesz job ETL, który automatycznie przetwarza dane.

---

## 2. Materiały Teoretyczne

### 2.1 Typy Widoków
- **Temporary View** - widok tymczasowy, dostępny tylko w bieżącej sesji Spark
- **Global Temporary View** - widok globalny, dostępny we wszystkich sesjach Spark
- **View (Materialized)** - widok trwały, zapisany w metastore

### 2.2 Databricks Jobs
- **Task** - pojedyncze zadanie (notebook, JAR, Python script)
- **Job** - zestaw tasków z zależnościami
- **Cluster** - zasoby obliczeniowe do wykonania jobu
- **Schedule** - harmonogram uruchamiania jobu

### 2.3 Best Practices
- Używaj widoków dla często używanych zapytań
- Dokumentuj widoki - dodawaj komentarze
- Monitoruj performance jobów
- Używaj retry policy dla jobów

---

## 3. Inicjalizacja Środowiska

Uruchom skrypt inicjalizacyjny dla per-user izolacji katalogów i schematów:

In [ ]:
%run ../../00_setup

## Konfiguracja

Import bibliotek i ustawienie zmiennych środowiskowych:

### 3.2 Konfiguracja Zmiennych

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

# Nazwy tabel z poprzednich warsztatów
CUSTOMERS_TABLE = f"{SILVER_SCHEMA}.customers_clean"
ORDERS_TABLE = f"{BRONZE_SCHEMA}.orders_workshop"
PRODUCTS_TABLE = f"{BRONZE_SCHEMA}.products_workshop"

# Nazwy widoków do utworzenia
CUSTOMER_ORDERS_VIEW = f"{SILVER_SCHEMA}.vw_customer_orders"
TOP_CUSTOMERS_VIEW = f"{SILVER_SCHEMA}.vw_top_customers"
PRODUCT_SALES_VIEW = f"{SILVER_SCHEMA}.vw_product_sales"

print("=== Konfiguracja ===" )
print(f"Silver Schema: {SILVER_SCHEMA}")
print(f"Tabela klientów: {CUSTOMERS_TABLE}")
print(f"Widoki do utworzenia:")
print(f"  - {CUSTOMER_ORDERS_VIEW}")
print(f"  - {TOP_CUSTOMERS_VIEW}")
print(f"  - {PRODUCT_SALES_VIEW}")

---

## 4. Widoki Tymczasowe

### 4.1 Przykład: Temporary View
Ta komórka jest już gotowa - przeanalizuj kod i uruchom go.

In [ ]:
# Przykład: Tworzenie widoku tymczasowego
# Widok będzie dostępny tylko w tej sesji Spark

customers_df = spark.table(CUSTOMERS_TABLE)
customers_df.createOrReplaceTempView("tmp_customers")

# Użyj widoku w zapytaniu SQL
result = spark.sql("""
    SELECT 
        city,
        COUNT(*) as customer_count
    FROM tmp_customers
    GROUP BY city
    ORDER BY customer_count DESC
    LIMIT 10
""")

print("=== Top 10 miast z najwięcej klientami ===")
display(result)

### 4.2 Zadanie: Global Temporary View

**TODO**: Uzupełnij kod, aby:
1. Utworzyć globalny widok tymczasowy dla tabeli zamówień
2. Wyświetlić statystyki zamówień z tego widoku

**Wskazówka**: Użyj `.createOrReplaceGlobalTempView()` i odwołuj się przez `global_temp.nazwa_widoku`

In [ ]:
# TODO: Utwórz globalny widok tymczasowy
orders_df = spark.table(ORDERS_TABLE)

# TODO: Uzupełnij kod
# orders_df.createOrReplaceGlobalTempView("gtmp_orders")

# TODO: Użyj widoku w zapytaniu SQL
# Wskazówka: SELECT ... FROM global_temp.gtmp_orders
result = spark.sql("""
    -- TODO: Napisz zapytanie, które policzy:
    -- - Całkowitą liczbę zamówień
    -- - Całkowitą wartość zamówień
    -- - Średnią wartość zamówienia
    -- - Minimalną i maksymalną wartość zamówienia
""")

print("=== Statystyki zamówień ===")
display(result)

---

## 5. Widoki Materialized (Trwałe)

### 5.1 Przykład: Widok Zamówień z Klientami
Ta komórka jest już gotowa - przeanalizuj kod i uruchom go.

In [ ]:
# Przykład: Widok łączący zamówienia z klientami
spark.sql(f"""
    CREATE OR REPLACE VIEW {CUSTOMER_ORDERS_VIEW}
    COMMENT 'Widok łączący dane zamówień z informacjami o klientach'
    AS
    SELECT 
        o.order_id,
        o.order_date,
        o.order_amount,
        o.order_status,
        c.customer_id,
        c.first_name,
        c.last_name,
        c.email,
        c.city,
        c.country
    FROM {ORDERS_TABLE} o
    INNER JOIN {CUSTOMERS_TABLE} c
        ON o.customer_id = c.customer_id
""")

print(f"Utworzono widok: {CUSTOMER_ORDERS_VIEW}")

# Sprawdź widok
result = spark.sql(f"SELECT * FROM {CUSTOMER_ORDERS_VIEW} LIMIT 10")
display(result)

### 5.2 Zadanie: Widok Top Klientów

**TODO**: Uzupełnij kod, aby utworzyć widok, który:
1. Policzy liczbę zamówień dla każdego klienta
2. Obliczy całkowitą wartość zamówień dla każdego klienta
3. Obliczy średnią wartość zamówienia
4. Pokaże tylko top 100 klientów (według wartości)

**Wskazówka**: Użyj `CREATE OR REPLACE VIEW` z `GROUP BY` i `ORDER BY ... LIMIT`

In [ ]:
# TODO: Utwórz widok top klientów
spark.sql(f"""
    CREATE OR REPLACE VIEW {TOP_CUSTOMERS_VIEW}
    COMMENT 'Top 100 klientów według wartości zamówień'
    AS
    SELECT 
        -- TODO: Dodaj kolumny:
        -- customer_id
        -- first_name, last_name
        -- COUNT(order_id) AS order_count
        -- SUM(order_amount) AS total_spent
        -- AVG(order_amount) AS avg_order_value
    FROM {CUSTOMER_ORDERS_VIEW}
    -- TODO: Dodaj GROUP BY
    -- TODO: Dodaj ORDER BY total_spent DESC
    -- TODO: Dodaj LIMIT 100
""")

print(f"Utworzono widok: {TOP_CUSTOMERS_VIEW}")

# Sprawdź widok
result = spark.sql(f"SELECT * FROM {TOP_CUSTOMERS_VIEW} LIMIT 10")
display(result)

### 5.3 Zadanie: Widok Sprzedaży Produktów

**TODO**: Uzupełnij kod, aby utworzyć widok sprzedaży produktów.
Zakładamy, że tabela zamówień ma kolumnę `product_id` i `quantity`.

**Wskazówka**: Połącz tabele zamówień i produktów, oblicz statystyki sprzedaży

In [ ]:
# TODO: Utwórz widok sprzedaży produktów
spark.sql(f"""
    CREATE OR REPLACE VIEW {PRODUCT_SALES_VIEW}
    COMMENT 'Statystyki sprzedaży produktów'
    AS
    SELECT 
        -- TODO: Dodaj kolumny:
        -- p.product_id, p.product_name, p.category
        -- COUNT(o.order_id) AS order_count
        -- SUM(o.quantity) AS total_quantity
        -- SUM(o.order_amount) AS total_revenue
    FROM {ORDERS_TABLE} o
    -- TODO: Dodaj JOIN z tabelą produktów
    -- TODO: Dodaj GROUP BY
    -- TODO: Dodaj ORDER BY total_revenue DESC
""")

print(f"Utworzono widok: {PRODUCT_SALES_VIEW}")

# Sprawdź widok
result = spark.sql(f"SELECT * FROM {PRODUCT_SALES_VIEW} LIMIT 10")
display(result)

---

## 6. Zapytania na Widokach

### 6.1 Przykład: Analiza Top Klientów
Ta komórka jest już gotowa - przeanalizuj kod i uruchom go.

In [ ]:
# Przykład: Zapytanie na widoku top klientów
result = spark.sql(f"""
    SELECT 
        customer_id,
        first_name || ' ' || last_name AS customer_name,
        order_count,
        ROUND(total_spent, 2) AS total_spent,
        ROUND(avg_order_value, 2) AS avg_order_value
    FROM {TOP_CUSTOMERS_VIEW}
    WHERE order_count >= 5
    ORDER BY total_spent DESC
    LIMIT 20
""")

print("=== Top 20 klientów z min. 5 zamówieniami ===")
display(result)

### 6.2 Zadanie: Analiza Segmentów Klientów

**TODO**: Napisz zapytanie SQL, które:
1. Użyje widoku `TOP_CUSTOMERS_VIEW`
2. Podzieli klientów na segmenty:
   - VIP: total_spent >= 5000
   - Premium: total_spent >= 2000 AND < 5000
   - Regular: total_spent < 2000
3. Policzy liczbę klientów w każdym segmencie
4. Obliczy średnią wartość zamówienia dla każdego segmentu

**Wskazówka**: Użyj `CASE WHEN` do utworzenia segmentów

In [ ]:
# TODO: Uzupełnij zapytanie SQL
result = spark.sql(f"""
    SELECT 
        -- TODO: Dodaj kolumnę segment używając CASE WHEN
        CASE 
            WHEN total_spent >= 5000 THEN 'VIP'
            WHEN total_spent >= 2000 THEN 'Premium'
            ELSE 'Regular'
        END AS segment,
        -- TODO: Dodaj COUNT(*) AS customer_count
        -- TODO: Dodaj AVG(avg_order_value) AS avg_order_value
        -- TODO: Dodaj SUM(total_spent) AS total_revenue
    FROM {TOP_CUSTOMERS_VIEW}
    -- TODO: Dodaj GROUP BY segment
    -- TODO: Dodaj ORDER BY total_revenue DESC
""")

print("=== Segmenty klientów ===")
display(result)

---

## 7. Zarządzanie Widokami

### 7.1 Przykład: Sprawdzanie Metadanych Widoku
Ta komórka jest już gotowa - przeanalizuj kod i uruchom go.

In [ ]:
# Przykład: Sprawdzenie metadanych widoku
print(f"=== Metadane widoku: {CUSTOMER_ORDERS_VIEW} ===")
spark.sql(f"DESCRIBE EXTENDED {CUSTOMER_ORDERS_VIEW}").show(truncate=False)

print(f"\n=== Definicja widoku ===")
spark.sql(f"SHOW CREATE TABLE {CUSTOMER_ORDERS_VIEW}").show(truncate=False)

### 7.2 Zadanie: Lista Wszystkich Widoków

**TODO**: Uzupełnij kod, aby:
1. Wyświetlić listę wszystkich widoków w schemacie SILVER
2. Dla każdego widoku wyświetlić: nazwę, typ (VIEW vs TABLE), komentarz

**Wskazówka**: Użyj `SHOW TABLES IN schema` i przefiltruj po typie

In [ ]:
# TODO: Wyświetl listę widoków
# Wskazówka: SHOW TABLES IN schema
result = spark.sql(f"""
    -- TODO: Napisz zapytanie, które wyświetli widoki w SILVER_SCHEMA
    -- Wskazówka: SHOW TABLES IN {SILVER_SCHEMA}
""")

# Przefiltruj tylko widoki (isTemporary = false)
views_only = result.filter(F.col("isTemporary") == False)

print(f"=== Widoki w schemacie: {SILVER_SCHEMA} ===")
display(views_only)

---

## 8. Przygotowanie Notebooka dla Joba

### 8.1 Przykład: Parametryzowany Notebook
Ta komórka demonstruje, jak utworzyć notebook, który może być uruchomiony jako job z parametrami.

In [ ]:
# Przykład: Widget do parametryzacji notebooka
# W Databricks możesz utworzyć widget, który pozwala przekazać parametry do notebooka

# Utwórz widget dla daty przetwarzania
dbutils.widgets.text("processing_date", "2025-01-31", "Data przetwarzania")
dbutils.widgets.dropdown("mode", "full", ["full", "incremental"], "Tryb przetwarzania")

# Odczytaj wartości parametrów
processing_date = dbutils.widgets.get("processing_date")
mode = dbutils.widgets.get("mode")

print(f"=== Parametry jobu ===")
print(f"Data przetwarzania: {processing_date}")
print(f"Tryb: {mode}")

### 8.2 Zadanie: ETL Pipeline

**TODO**: Uzupełnij kod pipeline'u, który:
1. Wczyta dane z tabel bronze
2. Wyczyści i przekształci dane
3. Zapisze wyniki do tabel silver
4. Odświeży widoki
5. Wyświetli statystyki przetwarzania

Ten kod będzie używany w jobie Databricks.

In [ ]:
# TODO: Implementuj pipeline ETL
from datetime import datetime

# Rozpocznij pipeline
start_time = datetime.now()
print(f"=== Pipeline ETL rozpoczęty: {start_time} ===")

# Krok 1: Wczytaj dane bronze
print("\n[1/5] Wczytywanie danych bronze...")
# TODO: Wczytaj tabele bronze
customers_bronze = spark.table(f"{BRONZE_SCHEMA}.customers_workshop")
orders_bronze = spark.table(f"{BRONZE_SCHEMA}.orders_workshop")

print(f"  - Klienci: {customers_bronze.count()} wierszy")
print(f"  - Zamówienia: {orders_bronze.count()} wierszy")

# Krok 2: Czyszczenie danych
print("\n[2/5] Czyszczenie danych...")
# TODO: Usuń duplikaty, obsłuż NULL, standaryzuj dane
customers_clean = (
    customers_bronze
    .dropDuplicates(["customer_id"])
    .withColumn("email", F.trim(F.lower(F.col("email"))))
    .withColumn("last_name", F.initcap(F.col("last_name")))
)

orders_clean = (
    orders_bronze
    .dropna(subset=["order_amount"])
    .fillna({"order_status": "Unknown"})
)

print(f"  - Klienci po czyszczeniu: {customers_clean.count()} wierszy")
print(f"  - Zamówienia po czyszczeniu: {orders_clean.count()} wierszy")

# Krok 3: Zapis do silver
print("\n[3/5] Zapis do silver schema...")
# TODO: Zapisz wyczyszczone dane do silver
(
    customers_clean
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(f"{SILVER_SCHEMA}.customers_clean")
)

(
    orders_clean
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(f"{SILVER_SCHEMA}.orders_clean")
)

print("  - Tabele zapisane do silver")

# Krok 4: Odśwież widoki (opcjonalnie recreate)
print("\n[4/5] Odświeżanie widoków...")
# TODO: Odśwież widoki jeśli to konieczne
# Wskazówka: W Delta Lake widoki są automatycznie aktualizowane
print("  - Widoki odświeżone")

# Krok 5: Statystyki
print("\n[5/5] Statystyki przetwarzania...")
end_time = datetime.now()
duration = (end_time - start_time).total_seconds()

print(f"\n=== Pipeline ETL zakończony ===")
print(f"Czas rozpoczęcia: {start_time}")
print(f"Czas zakończenia: {end_time}")
print(f"Czas trwania: {duration} sekund")
print(f"\nPrzetworzone rekordy:")
print(f"  - Klienci: {customers_clean.count()}")
print(f"  - Zamówienia: {orders_clean.count()}")

---

## 9. Konfiguracja Joba (Instrukcja)

### 9.1 Tworzenie Joba w Databricks UI

**Krok po kroku:**

1. **Przejdź do zakładki Jobs**
   - W menu bocznym wybierz "Workflows" → "Jobs"
   - Kliknij "Create Job"

2. **Skonfiguruj Task**
   - **Task name**: `etl_silver_pipeline`
   - **Type**: `Notebook`
   - **Source**: Wybierz ten notebook
   - **Cluster**: Wybierz istniejący cluster lub utwórz nowy

3. **Parametry (opcjonalnie)**
   - Dodaj parametry dla `processing_date` i `mode`
   - Przykład: `{"processing_date": "2025-01-31", "mode": "full"}`

4. **Harmonogram (opcjonalnie)**
   - **Schedule**: Cron expression, np. `0 2 * * *` (codziennie o 2:00)
   - **Timezone**: Wybierz swoją strefę czasową

5. **Retry Policy**
   - **Max retries**: 3
   - **Retry interval**: 60 sekund

6. **Notifications (opcjonalnie)**
   - Dodaj email dla powiadomień o sukcesie/błędzie

7. **Zapisz i uruchom**
   - Kliknij "Create" aby zapisać job
   - Kliknij "Run now" aby uruchomić job ręcznie

### 9.2 Monitorowanie Joba

Po uruchomieniu joba:
- Obserwuj status w zakładce "Runs"
- Sprawdź logi w "Logs"
- Analizuj metryki w "Metrics"
- Sprawdź outputy notebooka

---

## 10. Zadanie Podsumowujące

### 10.1 TODO: Kompleksowe Zadanie

**Zadanie**: Zaimplementuj kompletny proces:

1. Utwórz widok `vw_monthly_sales` pokazujący:
   - Rok i miesiąc
   - Liczbę zamówień
   - Całkowitą wartość sprzedaży
   - Średnią wartość zamówienia
   - Liczbę unikalnych klientów

2. Napisz zapytanie, które:
   - Użyje utworzonego widoku
   - Obliczy wzrost sprzedaży miesiąc do miesiąca (MoM growth)
   - Wyświetli trendy

3. Przygotuj ten kod do uruchomienia jako job

In [ ]:
# TODO: Implementuj zadanie podsumowujące

# Część 1: Utwórz widok monthly_sales
# TODO: Napisz CREATE OR REPLACE VIEW

# Część 2: Zapytanie z trendem MoM
# TODO: Użyj LAG() lub LEAD() do obliczenia wzrostu

# Część 3: Dodaj error handling i logging
# TODO: try-except, logging statystyk

print("Zadanie zakończone!")

---

## 11. Best Practices i Podsumowanie

### 11.1 Best Practices dla Widoków
1. **Nazywaj widoki prefiksem** - `vw_` dla łatwej identyfikacji
2. **Dodawaj komentarze** - dokumentuj cel widoku
3. **Używaj widoków dla logiki biznesowej** - enkapsulacja złożonych zapytań
4. **Monitoruj performance** - widoki mogą wpływać na wydajność
5. **Rozważ materialized views** - dla często używanych, kosztownych zapytań

### 11.2 Best Practices dla Jobs
1. **Parametryzuj notebooki** - używaj widgets
2. **Implementuj error handling** - try-except, retry policy
3. **Loguj wszystko** - starty, zakończenia, błędy, statystyki
4. **Monitoruj joby** - alerting, notifications
5. **Testuj przed wdrożeniem** - uruchom ręcznie i sprawdź wyniki
6. **Używaj idempotentnych operacji** - job można uruchomić wielokrotnie bez efektów ubocznych

### 11.3 Co osiągnąłeś?
- Stworzyłeś widoki tymczasowe i trwałe
- Stworzyłeś widoki biznesowe (top klienci, sprzedaż produktów)
- Napisałeś zapytania wykorzystujące widoki
- Przygotowałeś notebook do uruchomienia jako job
- Zaimplementowałeś prosty pipeline ETL
- Poznałeś best practices dla widoków i jobs

### 11.4 Następne kroki
- Delta Live Tables (DLT)
- Zaawansowane joby (multi-task workflows)
- Monitoring i alerting
- CI/CD dla Databricks

---

## 12. Czyszczenie Zasobów

In [ ]:
# Opcjonalnie: usuń widoki testowe
# spark.sql(f"DROP VIEW IF EXISTS {CUSTOMER_ORDERS_VIEW}")
# spark.sql(f"DROP VIEW IF EXISTS {TOP_CUSTOMERS_VIEW}")
# spark.sql(f"DROP VIEW IF EXISTS {PRODUCT_SALES_VIEW}")

# Usuń widoki tymczasowe
# spark.catalog.dropTempView("tmp_customers")
# spark.catalog.dropGlobalTempView("gtmp_orders")

# Usuń widgety
dbutils.widgets.removeAll()

print("Zasoby wyczyszczone!")
print("Warsztat zakończony!")